# Scikit-Learn LogisticRegression
Using IRIS_VIEW from DWC. This view has 150 records

## Install fedml-azure package

In [1]:
pip install fedml-azure --force-reinstall

Processing ./fedml_azure-1.0.0-py3-none-any.whl
  Using cached hdbcli-2.10.13-cp34-abi3-manylinux1_x86_64.whl (11.7 MB)
  Attempting uninstall: hdbcli
    Found existing installation: hdbcli 2.10.13
    Uninstalling hdbcli-2.10.13:
      Successfully uninstalled hdbcli-2.10.13
  Attempting uninstall: fedml-azure
    Found existing installation: fedml-azure 1.0.0
    Uninstalling fedml-azure-1.0.0:
      Successfully uninstalled fedml-azure-1.0.0
Note: you may need to restart the kernel to use updated packages.


## Import the libraries needed in this notebook

In [2]:
from fedml_azure import create_workspace
from fedml_azure import create_compute
from fedml_azure import create_environment
from fedml_azure import DwcAzureTrain

## Set up
### Creating a workspace. This takes a dictionary as input for parameter workspace_args.

Before running the below cell, ensure that you have a workspace and replace the subscription_id, resource_group, and workspace_name with your information.
https://docs.microsoft.com/en-us/azure/machine-learning/how-to-manage-workspace?tabs=python


In [3]:
#creation of workspace
workspace=create_workspace(workspace_args={
                                            "subscription_id": '<subscription-id>',
                                            "resource_group": '<resource-group>',
                                            "workspace_name": '<workspace_name>'
                                            }
)

Getting existing Workspace


### Creating a Compute Cluster. This takes the workspace, a compute_type, and compute_args.
https://docs.microsoft.com/en-us/azure/machine-learning/how-to-create-attach-compute-cluster?tabs=python

https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.compute.amlcompute.amlcompute?view=azure-ml-py

In [4]:
#creation of compute target
compute=create_compute(workspace=workspace,
                   compute_type='AmlComputeInstance',
                   compute_args={
                       'vm_size':"Standard_D3_v2",
                       'compute_name':'cpu-clu-log'
                   }
                )



Creating Compute_target
Found existing instance, using it.


### Creating an Environment. This takes the workspace, environment_type, and environment_args.

The fedml-azure pip package must be passed to the pip_packages key in the environment_args and 
to use scikit-learn, you must pass the name to conda_packages as well.

In this example, we are using a .yml for the environments dependencies and passing the file path to environment_arg's file_path key.

https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.environment(class)?view=azure-ml-py

https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments

In [5]:
#creation of environment
environment=create_environment(workspace=workspace,
                           environment_type='CondaSpecificationEnvironment',
                           environment_args={'name':'test-env-log',
                           'file_path': 'conda_dependency.yml'})

Creating Environment


## Now, let's train the model
### First, we need to instantiate the training class - this will assign the resources.

In [6]:
train=DwcAzureTrain(workspace=workspace,
                 environment=environment,
                 experiment_args={'name':'test-2'},
                 compute=compute)

Assigning Workspace
Creating Experiment
Assigning compute
Assigning Environment


### Then, we need to generate the run config. This is needed to package the configuration specified so we can submit a job for training. 

Before running the following cell, you should have a config.json file with the specified values to allow you to access to DWC. Provide this file path to config_file_path in the below cell.

You should also have the follow view IRIS_VIEW created in your DWC. To gather this data, please refer to https://www.kaggle.com/uciml/iris

https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.scriptrunconfig?view=azure-ml-py

In [7]:
#generating the run config
src=train.generate_run_config(config_file_path='dwc_configs/config.json',
                          config_args={
                                          'source_directory':'Scikit-Learn-Logistic-Regression',
                                          'script':'iris_train.py',
                                          'arguments':[
                                                        '--model_file_name','regression.pkl',
                                                        '--table_name', 'IRIS_VIEW',
                                                        '--table_size', '1']
                                          }
                            )

Generating script run config
Config file already exists in the script_directory Scikit-Learn-Logistic-Regression


### Submitting the job for training

In [ ]:
#submitting the training run
run=train.submit_run(src)

## Register the model for deployment

In [9]:
model=train.register_model(run=run,
                           model_args={'model_name':'sklearn_logReg_model',
                                       'model_path':'outputs/regression.pkl'},
                            resource_config_args={'cpu':1, 'memory_in_gb':0.5},
                            is_sklearn_model=True
                           )
print('Name:', model.name)
print('Version:', model.version)

Registering the model
Configuring parameters for sklearn model
Name: sklearn_logReg_model
Version: 5
